
### Librerías

In [76]:
import sys
import pandas as pd
import GetOldTweets3 as got
#from ipynb.fs.full.GetOldTweets import *
import sys
import matplotlib.pyplot as plt

### Configuración de parámetros 

In [61]:
pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000

### Ejecución de códigos auxiliares

In [135]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb
%run GetOldTweets.ipynb
%run NLP.ipynb

# Método Principal - TFM Data Science

In [ ]:
if __name__ == '__main__':
    # Definición de variables
    df_tweets, df_tweets_final = creacion_DataFrame()
    calendario = calendar()
    all_tweets = 1
    positivo, negativo, neutral, sentiment = creacion_Variables()
    
    #Acceso a Twitter
    api = acceso_Twitter() 
    
    #Lectura de ficheros para ir más rápido ubicados en CSVs
    if all_tweets == 0:
        df_tweets = read_all_Tweets() 
    else: 
        file = 'CSVs/info_tweets_el_corte_ingles_202003.csv'
        df_tweets = read_Tweets(file)
    
    df_tweets['valor'] = -10
    df_tweets['fecha'] = df_tweets['date'].str[:10]
    df_tweets['fecha'] = df_tweets['fecha'].str.replace('-','/')

    #Obtención por GetOldTweets
    #df_tweets = get_Tweets(calendario)
    

    #Pasando el algoritmo de NLP para obtener el análisis de sentimiento
    df_tweets_final = analisis_2(df_tweets, df_tweets_final)
    print(len(df_tweets_final))
    #Generación de CSVs para el CdM
    
    writeCSV(df_tweets_final,'CdM/Prueba_.xlxs')
    
    #Pintamos en un gráfico de tartas el sentimiento de los tweets
    """labels = ['Positivo ['+str(sentiment[0])+'%]','Neutral ['+str(sentiment[0])+'%]','Negativo ['+str(sentiment[2])+'%]']
    sizes = [sentiment[0],sentiment[1],sentiment[2]]
    colors =['yellowgreen','gold','green']
    patches, texts = plt.pie(sizes, colors=colors, startangle=90)
    plt.title('Opinión ECI - COVID')
    plt.axis('equal')
    plt.show()"""    
    

In [153]:
df_tweets['fecha'] = df_tweets['date'].str[:10]
df_tweets['fecha']=df_tweets['fecha'].str.replace('-','/')
df_tweets

,id,date,username,text,geo,valor,fecha
0,1244775324367323142,2020-03-30 23:55:38+00:00,conchymor,"En serio empresas como el corte inglés, indite...",NaN,-10,2020/03/30
1,1244773416835715072,2020-03-30 23:48:04+00:00,Nerea_Blake,Al día siguiente fui a El Corte Inglés y me co...,NaN,-10,2020/03/30
2,1244772598589816836,2020-03-30 23:44:48+00:00,sarureview,"Pero, a ver, una cosa es una expresión propia ...",NaN,-10,2020/03/30
3,1244770046913712128,2020-03-30 23:34:40+00:00,Julicanales,El confinamiento ha retratado la penosa distri...,NaN,-10,2020/03/30
4,1244768892146388992,2020-03-30 23:30:05+00:00,daniserna1983,Es que para ella siempre será mejor una donaci...,NaN,-10,2020/03/30
...,...,...,...,...,...,...,...
10463,1233919894380519426,2020-03-01 01:00:02+00:00,compradiccion,El reloj inteligente low cost de Xiaomi ahora ...,NaN,-10,2020/03/01
10464,1233912517832908802,2020-03-01 00:30:43+00:00,pulsosanitario,El Corte Inglés retira sus yogures de Lactalis...,NaN,-10,2020/03/01
10465,1233906756620820481,2020-03-01 00:07:50+00:00,ligasordidaf1,Recuerdo a esta tipeja en una conferencia en E...,NaN,-10,2020/03/01
10466,1233904930907398145,2020-03-01 00:00:35+00:00,DavidMeCa10,Bar El Temple enfrente de la puerta de accesos...,NaN,-10,2020/03/01
